<a href="https://colab.research.google.com/github/anjaleeDS/DAT6-git-lab/blob/master/PandasBOOTCAMP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

tips_data = sns.load_dataset("tips") # sns has loaded as a panda dataframe (pd) 

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Notes

- pandas is like a spreadsheet
- a series == a column in pandas
- dependent on RAM, parallel processing on python

## Tips Dataset Exploration

In [0]:
tips_data.columns
tips_data.shape # tells the number of rows and columns
tips_data.head() # defaults to 5. this is a named argument
tips_data.describe() # only shows the numeric columns and some of the information about 

,total_bill,tip,size
count,244.000000,244.000000,244.000000
mean,19.785943,2.998279,2.569672
std,8.902412,1.383638,0.951100
min,3.070000,1.000000,1.000000
25%,13.347500,2.000000,2.000000
50%,17.795000,2.900000,2.000000
75%,24.127500,3.562500,3.000000
max,50.810000,10.000000,6.000000


In [0]:
tips_data.isnull() # same as isna! checks df for any NaNs
tips_data.info() # this infers types and also counts the null values by showing the "Non-Null Count"
tips_data.tip # is the same as tips_data['tip']

In [0]:
tips_total_df = tips_data[['tip', 'total_bill']]
tips_data[['tip', 'total_bill']].describe() # chaining in practice
tips_total_df.sum() # gives the sum of the columns


In [0]:
tips_total_df.total_bill.iloc[4]    # index location 4
np.log(tips_total_df.total_bill)    # get log for all the values in tips_total_df

tips_total_df.total_bill.apply(np.log) # this is the same is np.log(tips_total_df.total_bill) 
    # but the .apply is the important part
tips_total_df.apply(np.log) # APPLY CAN PASS ANY COLUMNS OR ALL COLUMNS THROUGH 
    # TO THE FUNCTION WHERE AS IF YOU USE np.log IT'S ONLY APPLIED TO ONE COLUMNN 

In [0]:
tips_total_df.total_bill.iloc[3] = 0    # setting values within the df. BEWARE!!! DON'T OVERWRITE
tips_total_df_copied = tips_total_df.copy  # GOOD HOUSEKEEPING - make a copy to 
    # be sure you don't change anything in the original values

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [0]:
# creating new column
tips_total_df['tip_frac'] = tips_total_df.tip / tips_total_df.total_bill
tips_total_df.head()
tips_total_df.apply(np.isfinite) # pandas doesnt have the check inf function 
  # but numpy does, so you just use the xyz_df.APPLY(NUMPY.FUNCTION)

In [0]:
tips_total_df[tips_total_df.tip_frac.apply(np.isfinite)] # only show the values that is FINITE -- NO MORE [3]!

In [0]:
tips_total_df = tips_total_df.reset_index() # get the [3] back

## Stock data set

In [0]:
from vega_datasets import data

stocks_data = data.stocks()
stocks_data.head()
stocks_data.info()
stocks_data.symbol.value_counts() # just get the column/series symbol and the counts
# doing a stocks_data.describe doesn't make sense because the symbol-values are mixed symbols

In [0]:
# pivot tables: pivot requires one value per day with NO aggregations. pivot_table can give aggregates 
stocks_data_pivoted = stocks_data.pivot_table(index='date', columns='symbol', values='price', aggfunc='mean')
stocks_data_pivoted.sort_index(ascending=False) # reverse chron since chron is the index=date
stocks_data_pivoted.sort_values('AAPL', ascending=False)  # sort_values allows for selection and sort

In [0]:
stocks_data_pivoted.fillna(method='ffill')
stocks_data_pivoted

symbol,AAPL,AMZN,GOOG,IBM,MSFT
date,,,,,
2000-01-01,25.94,64.56,NaN,100.52,39.81
2000-02-01,28.66,68.87,NaN,92.11,36.35
2000-03-01,33.95,67.00,NaN,106.11,43.22
2000-04-01,31.01,55.19,NaN,99.95,28.37
2000-05-01,21.00,48.31,NaN,96.31,25.45
...,...,...,...,...,...
2009-11-01,199.91,135.91,583.00,125.79,29.27
2009-12-01,210.73,134.52,619.98,130.32,30.34
2010-01-01,192.06,125.41,529.94,121.85,28.05


In [0]:
stocks_data_pivoted = stocks_data.pivot_table(index='date', columns='symbol', values='price', aggfunc='mean')

stocks_data_pivoted_pct_change = stocks_data_pivoted.pct_change()
# stocks_data_pivoted.diff(1)
stocks_data_pivoted_pct_change
# st_pct_change = stocks_data_pivoted.pct_change()
stocks_cum_sum =stocks_data_pivoted_pct_change.cumsum()

In [0]:
# plot from the beginning of GOOG is not a nan
stocks_data_pivoted[stocks_data_pivoted['GOOG'] > 0]
# OR
stocks_data_pivoted[stocks_data_pivoted['GOOG'].isna() != True]

### MELT DATA

In [0]:
# how do we go back to original format? IT'S CALLED A FLATTENED DATA STRUCTURE
# tableau works better when it's flattened
# hue also only works with flattened data
# flatted aka "MELTED" for pandas

# MELT DATA!

# first make the date index it's own column so we can get index back
stocks_data_pivoted = stocks_data_pivoted.reset_index() # this will add a new index to the df
stocks_data_pivoted.head()

In [0]:
stocks_data_pivoted.melt(id_vars='date', value_name='price', value_vars=['AAPL', 'AMZN'])

,date,symbol,price
0,2000-01-01,AAPL,25.94
1,2000-02-01,AAPL,28.66
2,2000-03-01,AAPL,33.95
3,2000-04-01,AAPL,31.01
4,2000-05-01,AAPL,21.00
...,...,...,...
241,2009-11-01,AMZN,135.91
242,2009-12-01,AMZN,134.52
243,2010-01-01,AMZN,125.41
244,2010-02-01,AMZN,118.40


In [0]:
# # DELETING COLUMNS

# del stocks_data_pivoted['level_0']
# del stocks_data_pivoted['symbol']
stocks_data_pivoted = stocks_data_pivoted.reset_index(drop=True)
stocks_data_pivoted

symbol,index,date,AAPL,AMZN,GOOG,IBM,MSFT
0,0,2000-01-01,25.94,64.56,NaN,100.52,39.81
1,1,2000-02-01,28.66,68.87,NaN,92.11,36.35
2,2,2000-03-01,33.95,67.00,NaN,106.11,43.22
3,3,2000-04-01,31.01,55.19,NaN,99.95,28.37
4,4,2000-05-01,21.00,48.31,NaN,96.31,25.45
...,...,...,...,...,...,...,...
118,118,2009-11-01,199.91,135.91,583.00,125.79,29.27
119,119,2009-12-01,210.73,134.52,619.98,130.32,30.34
120,120,2010-01-01,192.06,125.41,529.94,121.85,28.05
121,121,2010-02-01,204.62,118.40,526.80,127.16,28.67


## Sales Data for join - DO THESE ON THE SQL SERVER WHEN POSSIBLE
- join and merge is expensive in pandas
- merge is more expensive than join since join uses index

In [0]:
sales_data = pd.read_csv('https://raw.githubusercontent.com/cjflanagan/sample_data/master/sales%20data-set.csv')
stores_list = pd.read_csv('https://raw.githubusercontent.com/cjflanagan/sample_data/master/stores%20data-set.csv')

sales_data.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,05/02/2010,24924.50,False
1,1,1,12/02/2010,46039.49,True
2,1,1,19/02/2010,41595.55,False
3,1,1,26/02/2010,19403.54,False
4,1,1,05/03/2010,21827.90,False


In [0]:
stores_list.head()

,Store,Type,Size
0,1,A,151315
1,2,A,202307
2,3,B,37392
3,4,A,205863
4,5,B,34875


In [0]:
# join will use the index and merged you can specify a column name
# merge is more flexible if you don't have you use the index

sd_merged = sales_data.merge(stores_list, how='left', left_on='Store', right_on='Store')
sd_merged

,Store,Dept,Date,Weekly_Sales,IsHoliday,Type,Size
0,1,1,05/02/2010,24924.50,False,A,151315
1,1,1,12/02/2010,46039.49,True,A,151315
2,1,1,19/02/2010,41595.55,False,A,151315
3,1,1,26/02/2010,19403.54,False,A,151315
4,1,1,05/03/2010,21827.90,False,A,151315
...,...,...,...,...,...,...,...
421565,45,98,28/09/2012,508.37,False,B,118221
421566,45,98,05/10/2012,628.10,False,B,118221
421567,45,98,12/10/2012,1061.02,False,B,118221
421568,45,98,19/10/2012,760.01,False,B,118221


In [0]:
sd_merged.groupby('Type')['Weekly_Sales'].mean() # group by the type on the average (mean) weekly sales


Type
A    20099.568043
B    12237.075977
C     9519.532538
Name: Weekly_Sales, dtype: float64

In [0]:
# This is most common to use one function with multiple columns
sd_merged.groupby(['Type','IsHoliday'])['Weekly_Sales'].mean()

Type  IsHoliday
A     False        20008.746759
      True         21297.517824
B     False        12153.067752
      True         13346.164062
C     False         9518.528116
      True          9532.963131
Name: Weekly_Sales, dtype: float64

In [0]:
sd_merged_complex_index = sd_merged.groupby(['Type','IsHoliday']).agg({'Weekly_Sales':['mean', 'std', 'std']})
sd_merged_complex_index

Weekly_Sales                            
                        mean           std           std
Type IsHoliday                                          
A    False      20008.746759  26107.047984  26107.047984
     True       21297.517824  30264.671745  30264.671745
B    False      12153.067752  16558.647788  16558.647788
     True       13346.164062  24128.821137  24128.821137
C    False       9518.528116  15975.155545  15975.155545
     True        9532.963131  16123.779035  16123.779035

In [0]:
sd_merged_complex_reindex = sd_merged_complex_index.reset_index()

NameError: ignored